**Функции**

Функция - фрагмент программного кода, к которому можно обратиться из другого места программы

В Python функции задаются ключевым словом def. Например, вот пример функции, вычисляющей факториал, и функции, вычисляющей n-ое числи фибоначчи

In [19]:
def say_hello():
    print ("Hello")
    
say_hello()

Hello


In [20]:
def add(a, b):
    return a + b

print (add(3,4))

7


In [21]:
def factorial(n):
    acc = 1
    for i in range(1,n + 1):
        acc *= i
    return acc

def fibonacci(n):

    f0, f1 = 0, 1
    for i in range(n):
        f0, f1 = f1, f0 + f1
    return f0

In [22]:
print (factorial(5))
print (fibonacci(5))

120
5


**Анонимные функции**


![man](man.jpg)


Часто в коде возникает необходимость написать какую-то простую функцию, которая будет использована единожды и затем никак не будут использоваться. Создание отдельной функции для этого добавляет коду громоздкость и часто ухудшает понимание написанного. 
Для этих целей в Python введены **lambda-функции**. 

In [9]:
def add5(x):
    return x + 5

add5 = lambda x : x + 5

Обычно используется в функциях, принимающих функцию как аргумент.
Например - отсортировать ключи словаря по соответствующим им значениям

In [12]:
dt = {"a" : 10, "c" : 53, "d" : 31, "e" : 42}
keys = list(dt)
keys.sort(key = lambda x : dt[x])
print (keys)

['a', 'd', 'e', 'c']


Или в map (будет подробно разобран через лекцию)

In [14]:
numbers_str = "1 3 4 77 82"
numbers_lst = list(map(int, numbers_str.split()))
print (numbers_lst)

[1, 3, 4, 77, 82]


**ВАЖНО**

Lambda-функции не дают преимущества в скорости выполнения. Они просто удобный способ записи.

**Рекурсия**

![recursion](recursion.png)

Python поддерживает механизм рекурсии - вызов функцией самой себя. Перепишем, например, предыдущие функции в рекурсивной форме

In [ ]:
def factorial_rec(n):
    if n <= 1:
        return 1
    return n * factorial_rec(n - 1)

def fibonacci_rec(n):
    if n < 0:
        return 0
    
    if n == 0:
        return 0
    if n == 1:
        return 1
    
    return fibonacci_rec(n - 1) + fibonacci_rec(n - 2)

In [ ]:
print (factorial_rec(5))
print (fibonacci_rec(5))

С одной стороны, рекурсия облегчает работу серым клеточкам. БОльшая часть алгоритмов легче формулируется в виде рекурсии, чем в других видах. 

Есть и минусы. И они значительные. Во-первых, реализация рекурсии в любом языке требует дополнительных затрат по сравнению с императивным подходом.

In [38]:
%%timeit
factorial_rec(1000)

403 µs ± 15.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [39]:
%%timeit
factorial(1000)

274 µs ± 12.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [42]:
%%timeit
fibonacci_rec(20)

3.7 ms ± 133 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [43]:
%%timeit
fibonacci(20)

1.19 µs ± 39.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


 Во-вторых - неправильно написанная рекурсия приводит к зацикливанию программы. 

In [ ]:
def factorial_rec_wrong(n):
    # There is no recursion break condition here
    return n * factorial_rec_wrong(n - 1)

В-третьих - в python по умолчанию размер "[стэка вызовов](https://www.youtube.com/watch?v=ygK0YON10sQ)" , в котором, в первом приближении, хранится информация о исполняющихся в данный момент функциях, ограничена. Потому пакет, использующий рекурсию и нормально работающий на одних данных, может падать на других. 
![image_1](recursion_stack_1.png)
![image_2](recursion_stack_2.png)

In [ ]:
factorial_rec(10000) # outputs recursion error

Часть этих проблем можно решать, таки напрягая серые клеточки, возможно, в меньшей степени. 
У fibonacci есть две проблемы - то, что стэк, используемый нами - глобальный, и то, что мы повторяем многие вычисления множество раз. 

![fibonacci](fibonacci_recursion.png)

Исправим сначала первую проблему. Это делается легко.

In [89]:
def fibonacci_rec_no_limit(n):
    stack = []
    stack.append(n)
    acc = 0
    while stack:
        fib_ind = stack.pop()
        if fib_ind == 0:
            pass
        elif fib_ind == 1:
            acc += 1
        else:
            stack.append(fib_ind - 1)
            stack.append(fib_ind - 2)
    return acc

In [82]:
fibonacci_rec(10000) # outputs recursion error

RecursionError: maximum recursion depth exceeded in comparison

In [83]:
fibonacci_rec_no_limit(10000) # do not outputs but there is a SMALL problem with asymptotic

KeyboardInterrupt: 

И у нас проблемы с производительностью по сравнению с исходной версией (куда уж больше)

In [90]:
%%timeit
fibonacci(20)

1.2 µs ± 50.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [91]:
%%timeit
fibonacci_rec(20)

3.61 ms ± 169 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [92]:
%%timeit
fibonacci_rec_no_limit(20)

5.13 ms ± 188 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Вообще говоря, для чисел фибоначчи можно не так уж сильно заботиться об исчерпании стэка (маловероятно). Главная проблема 

![fibonacci](fibonacci_recursion.png)

Для ее решения сначала разберем аргументы по-умолчанию в функциях. 

Достаточно часто встречается ситуация, когда большинство предпочтительных аргументов для функции известно (например, вы знаете, что ваш алгоритм лучше всего запускать с lambda = 0.73, а n_samples = 143). В таком случае, конечно, можно написать в комментариях к своей функции, но лучше воспользоваться аргументами по-умолчанию. 

In [99]:
import random

def make_random_sequence(length=200, alphabet="ATGC"):
    seq_lst = [] # it's better to use here list comprehensions
    for i in range(length):
        seq_lst.append(random.choice(alphabet))
    seq = "".join(seq_lst) # it is better than appending symbols directly to seq, why? 
    return seq

Также часто вы просто хотите задать поведение функции по-умолчанию/наиболее частое поведение. 

In [103]:
def login(username="anonymous", password=None):
    """Some action"""
    pass

# the can call function in different ways
login("root", "ujdyzysqgfhjkm") 
login("guest")
login()
# Also you can specify the name of argument
login(password="nobody@mail.com") 

Можно комбинировать обязательный аргументы с аргументами, заданными по умолчанию. При этом обязательные должны идти первыми.

In [105]:
def write_random_fasta(out_file_path, name="random", length=200, alphabet="ATGC"):
    out_file = open(out_file_path, "w") # it is better to use with-construction here
    seq = make_random_sequence(length=length, alphabet=alphabet)
    out_file.write(">{}\n".format(name))
    out_file.write("{}\n".format(seq))
    out_file.close()

In [106]:
write_random_fasta("random.fasta")

С аргументами по-умолчанию есть одна опасность - они "закрепляются" за функцией в момент создания. Если ваш аргумент по-умолчанию - неизменяем (число, строка), то все хорошо. Если же это изменяемый объект, то возникают интересные ситуации

Рассмотрим, например, такую функцию. Если ей переданы оба аргумента, то она добавляет el к lst и возвращает измененный lst. Если только первый - то вовзращает [el]. По крайней мере, ожидается, что она будет работать так.

In [114]:
def add_to_list(el, lst = []):
    lst.append(el)
    return lst

Однако тут возникает проблема. Если запустить последующую учейку с кодом несколько раз, то результаты будут становиться все страньше и страньше.

In [115]:
print (add_to_list(5, [1,2,3])) # OK
print (add_to_list(5, [])) # OK
print (add_to_list(5)) # OK
print (add_to_list(5)) # WHAT???

[1, 2, 3, 5]
[5]
[5]
[5, 5]


![suprise](surprise_dexter.gif)

Дело в том, что значение по-умолчанию создавалось один раз. И вначала оно равно пустому списку. Но в третьем случае мы добавляем в этот список элемент. И после возвращения из функции изменения не пропадают. Потому на следующем вызове элемент добавляется не к пустому списку, а к списку, содержащему один элемент. И так далее. 
Как это лечить? - не использовать в значениях по-умолчанию изменяемые объекты (списки, словари и т.д), а использовать None

In [117]:
def add_to_list_wsmf(el, lst = None):
    if lst is None:
        lst = []
    lst.append(el)
    return lst

In [119]:
print (add_to_list_wsmf(5, [1,2,3])) # OK
print (add_to_list_wsmf(5, [])) # OK
print (add_to_list_wsmf(5)) # OK
print (add_to_list_wsmf(5)) # Still OK

[1, 2, 3, 5]
[5]
[5]
[5]


Иногда же такое поведение желательно - если у вас функция, которая для одинакового набора аргументов дает одинаковые значения, при этом вычисление ее долгое, то почему бы не хранить эти значения каким-либо образом

Вернемся теперь к задаче с числами Фибоначчи. Проблема здесь в том, что для вычисления Fn-числа фибоначчи, значение многих предыдущих чисел Фибоначчи будет вычислено множество раз. Воспользуемся разобранной выше особенностью поведения аргументов по-умолчанию, чтобы хранить уже вычисленные значения. Хранение уже вычисленных значений для рекурсивных функций называется мемоизацией (проверить термин) и используется довольно часто.  

![fibonacci](fibonacci_recursion.png)

Глобальные, локальные и нелокальные переменные

Напоминание - в Python выражение a = 5 означает как и объявление переменной a, так и присвоение ей значения 5, если она уже была объявлена.

In [158]:
# variable is created here
result = "Outer"

def func():
    # new variable is created here
    result = "Inner"
    print (result)

func()
print (result) 

Inner
Outer


Каким образом код, приведенный выше, понимает, что result вне функции и result в функции - разные и понимает, что и когда использовать. 

В Python есть специальный принцип, по которому происходит разрешение имен переменных - LEGB Rule

Для начала усложним код еще больше. В Python можно делать вложенные функции. Сделаем. 

In [159]:
### code1 variable is created here
result = "Global"

def enclosing_func():
    ### code2 variable is created here
    result = "Enclosing"
    def func():
        ### code3 variable is created here
        result = "Inner"
        print (result)
    ### code 4
    func()
    print (result)

## code 5
enclosing_func()
print (result) 

Inner
Enclosing
Global


Важно понимать, что имя функции - тоже переменная
Теперь само правило.
Разрешение имени переменной в функции производится в следующем порядке (если переменная найдена, то процесс прекращается):

1) **L. Local** - поиск среди всех переменных, объявленных в функции. - _code3_

2) **E. Enclosing** - переменные в локальной области видимости функции (если такая есть), внутри которой объявлена данная - _code2_ и _code4_

3) **G. Global** - переменные, которые заданы на верхнем уровне файла/модуля. - _code1_ и _code5_

4) **B. Built-in** - стандартные переменные Python (range, list и т.д) 


Иногда мы хотим уметь модифицировать функцией значение глобальной переменной. Это редко оправданно, но нужно знать, как это сделать)

Для того, чтобы явно указать Python, что мы хотим не создать новую переменную в функции, а использовать глобальную используется ключевое слово **global**.

In [161]:
### code1 variable is created here
result = "Global"

def enclosing_func():
    ### code2 variable is created here
    global result
    result = "Enclosing"
    def func():
        ### code3 variable is created here
        result = "Inner"
        print (result)
    ### code 4
    func()
    print (result)

## code 5
enclosing_func()
print (result) 

Inner
Enclosing
Enclosing


Важно понимать, что код функции просматривается целиком. Потому такой код будет выдавать ошибку 

In [184]:
n_samples = 100

def get_n_samples():
    if n_samples is not None:
        return n_samples
    else:
        n_samples = 50
        return n_samples
    
get_n_samples() # outputs unbound local error

UnboundLocalError: local variable 'n_samples' referenced before assignment

В то время как его близкий аналог - нет

In [185]:
n_samples = 100

def get_n_samples():
    if n_samples is not None:
        return n_samples
    else:
        return 50
    
get_n_samples()

100

Иногда мы хотим иметь возможность также сказать во вложенной функции, что мы хотим использовать в ней переменную из окружающей ее функции. Для этого используется ключевое слово **nonlocal** (это название приходит из языков функционального программирования)

In [178]:
### code1 variable is created here
result = "Global"

def enclosing_func():
    ### code2 variable is created here
    result = "Enclosing"
    def func():
        nonlocal result
        ### code3 variable is created here
        result = "Inner"
        print (result)
    ### code 4
    func()
    print (result)

## code 5
enclosing_func()
print (result) 

Inner
Inner
Global


Теперь приведем несколько примеров того, когда это нужно. Простейший пример в случае с глобальными переменными - когда вам нужно именно во время подключения модуля определять какие-то переменные

In [179]:
# some module file

NUMBER_OF_AVALIABLE_PROCS = 0
VERY_IMPORTANT_LIBRARY_PATH = ""

def find_number_of_procs():
    return 4

def get_very_important_library_path():
    return "/usr/lib/crazymonkey.dll"

def init():
    global NUMBER_OF_AVALIABLE_PROCS
    global VERY_IMPORTANT_LIBRARY_PATH
    NUMBER_OF_AVALIABLE_PROCS = find_number_of_procs()
    VERY_IMPORTANT_LIBRARY_PATH = get_very_important_library_path()

init()
print (NUMBER_OF_AVALIABLE_PROCS)
print (VERY_IMPORTANT_LIBRARY_PATH)

4
/usr/lib/crazymonkey.dll


В случае с nonlocal нам необходимо для начала понять, что функция в Python - это тоже объект.

Мы можем присваивать ее переменной

In [180]:
def add_two(a, b):
    return a + b

adder = add_two # we can assign function to variable 
print (add_two(3,4))
print (adder(3, 4))

7
7


Мы можем вовзвращать функцию из функции

In [182]:
# creates function, which given argument a return function, which adds a to his argument b
def adder_creator(a):
    def adder(b):
        return a + b
    return adder

add5 = adder_creator(5)
add10 = adder_creator(10)
number = 7
print ("Add5", add5(number))
print ("Add10", add10(number))

Add5 12
Add10 17


Мы можем посылать функцию как аргумент

In [204]:
def compose(func1, func2, x):
    return func1(func2(x))

add5 = adder_creator(5)
add10 = adder_creator(10)

print (compose(add5, add10, 0))

15


Что происходит в коде выше?. В тот момент, когда создается функция adder, она получает информацию, что переменная a хранится в окружающей ее функции. И она всегда (во время жизни программы), может к ней обратиться, так как Python понимает, что на переменную a существует ссылка из функции adder и потому не удаляет ее. Это явление называется **захватом переменной**

![you_variable_is_mine](your-variable-is-mine.jpg)

Теперь можно привести пример того, когда нам нужно использование nonlocal. Допустим, мы хотим в каком-то сложном коде для каждого объекта заводить id. Есть несколько подходов к решению данной проблемы и один из них - использование **nonlocal**

In [198]:
def get_counter(start_id = 0):
    count = start_id - 1
    def counter():
        nonlocal count
        count += 1
        return count
    return counter

counter1 = get_counter(1)
print (counter1())
print (counter1())
print (counter1())
counter10 = get_counter(10) 
print (counter10())
print (counter10())
print (counter10())

print (counter1 == counter1)
print (counter1 == counter10) # the returned functions are DIFFERENT
print (get_counter(1) == get_counter(1))  # they are still DIFFERENT

1
2
3
10
11
12
True
False
False


То есть мы используем **nonlocal** тогда, когда, по какой-то причине, хотим дать нашей функции возможность изменять внешнюю по отношению к ней переменную, не делая при этом эту переменную глобальной. 

Здесь мы подходим к важной теме, которую начнем сегодня, а именно - реализация функционального программирования в Python.

Что же такое функциональное программирование? Очень грубо, это трактовка любого алгоритма как результата применения группы функций друг к другу и к каким-либо объектам. Часто такой подход позволяет сделать что-либо очень изящно, часто приводит к нечитаемумому коду. Кому что нравится) 

То, что мы использовали в нескольких предыдущих примерах в общем виде называется **замыканием** 

**Замыкание** - это функция вместе с привязанной к ней совокупностью данных 
@Дэвид Мертц

Действительно, в случае get_counter мы "привязываем" к создаваемой в ней функции counter переменную типа int - count. В дальнейшем counter спокойно обращается к этой переменной и может даже менять ее значение. 

По-аналогии можно сделать конструктор функций, умножающих на заданное число

In [203]:
def multiplier( n ):    # multiplier возвращает функцию умножения на n
    def mul( k ):
        return n * k
    return mul
 
mul3 = multiplier( 3 )  # mul3 - функция, умножающая на 3
print( mul3( 3 ), mul3( 5 ) )

9 15


А что если мы хотим написать функцию для отладки, которая принимает определенную функцию, а возвращает функцию, которая ведет себя так же, как и исходная, но печатает при старте функции "Function started", а при завершении ее работы - "Function ended"
Если мы знаем, что целевая функция принимает только, допустим, один аргумент, то все просто

In [209]:
def fibonacci(n):

    f0, f1 = 0, 1
    for i in range(n):
        f0, f1 = f1, f0 + f1
    return f0

def simple_wrapper(func):
    def decorated_func(n):
        print ("Function started")
        result = func(n)
        print ("Function ended")
        return result
    return decorated_func

print ("Real", fibonacci(10))
decorated_fibonacci = simple_wrapper(fibonacci)

print ("Decorated", decorated_fibonacci(10))

Real 55
Function started
Function ended
Decorated 55


Однако, в таком случае нам придется писать для каждого набора аргументов соответсвующую функцию, выполняющую требуемую функциональность. Это не очень впечатляет

В Python существует специальный механизм для случаев, когда наша функция может принять неизвестное заранее количество аргументов. Разберем его

Если наша функция принимает произвольное количество позиционных аргументов, то мы можем отразить это за счет **\*args**. 

Например, напишем функцию, находяющую произведение любого количества переданных ей чисел

In [212]:
def multi_product(*args):
    product = 1
    for comp in args:
        product *= comp
    return product

print (multi_product(1, 3, 3))
print (multi_product(1, 3, 3, 7))

9
63


Если наша функция принимает произвольное количество именованных аргументов (вида key=value), 
то мы можем использовать **\*\*kwargs**. 

Например, допустим мы передаем функции набор key и value, а она должна напечатать строку с key и факториалом от value

In [395]:
def multi_print_factorial(**kwargs):
    for key, value in kwargs.items():
        print (key, factorial(value))
        
multi_print_factorial(one=1, two=2, three=3)

one 1
two 2
three 6


Очевидно, все типы аргументов можно комбинировать

In [224]:
def some_complex_func(a, b, t=5, *args, **kwargs):
    print (a, b, t)
    print (args)
    print (kwargs)

some_complex_func(5, 5, 100, "cat", "dog", key="value")

5 5 100
('cat', 'dog')
{'key': 'value'}


Вернемся теперь к нашей задаче:
     написать функцию для отладки, которая принимает определенную функцию, а возвращает функцию, которая ведет себя так же, как и исходная, но печатает при старте функции "Function started", а при завершении ее работы - "Function ended".

In [268]:
def print_wrapper(func):
    def decorated_func(*args, **kwargs):
        print ("Function started")
        result = func(*args, **kwargs)
        print ("Function ended")
        return result
    return decorated_func

In [269]:
def find_gcd(a, b):
    if a < b:
        a, b = b, a
    if b == 0:
        return a
    return find_gcd(b, a % b)

print ("Greatest common divisor of 21 and 63 is", find_gcd(21, 63))

decorated_gcd = print_wrapper(find_gcd)
print ("Greatest common divisor of 21 and 63 is", decorated_gcd(21, 63))

Greatest common divisor of 21 and 63 is 21
Function started
Function ended
Greatest common divisor of 21 and 63 is 21


Функция, подобная той, что мы написали, называется **декоратором**. Для декораторов существует упрощенный синтаксис (однако он приведет к тому, что исходную функцию восстановить сложно)

In [270]:
@print_wrapper
def find_gcd(a, b):
    if a < b:
        a, b = b, a
    if b == 0:
        return a
    return find_gcd(b, a % b)

print ("Greatest common divisor of 21 and 63 is", find_gcd(21, 63))

Function started
Function started
Function ended
Function ended
Greatest common divisor of 21 and 63 is 21


Почему в этом примере функция написала о своем старте и выходе два раза?

В стандартной библиотеке Python есть множество декораторов, с которыми вы познакомитесь позже.

Декоратору непросто передать дополнительные аргументы, т.к он принимает первым аргументом функцию. На самом деле, для этого надо сделать не декоратор, а функцию, возвращающую декоратор

Напишем, например, декоратор, который будет писать в лог текущее время и дату и свое имя

In [284]:
def log_wrapper(name):
    import datetime
    def real_decorator(func):
        def decorated_func(*args, **kwargs):
            now = datetime.datetime.now()
            print ("{}, Date: {}, Function started".format(name, now))
            result = func(*args, **kwargs)
            now = datetime.datetime.now()
            print ("{}, Date: {}, Function ended".format(name, now))
            return result
        return decorated_func
    return real_decorator

In [287]:
@log_wrapper("Simple logger")
def fibonacci(n):
    f0, f1 = 0, 1
    for i in range(n):
        f0, f1 = f1, f0 + f1
    return f0

res = fibonacci(1000000)

Simple logger, Date: 2018-02-16 17:48:22.443671, Function started
Simple logger, Date: 2018-02-16 17:48:32.752694, Function ended


Для особо настойчивых привожу код, взятый из ответа со StackOveflow (https://stackoverflow.com/questions/5929107/decorators-with-parameters, пользователь Dacav). Если вы его поняли, то можете считать, что вы поняли декораторы. Он позволяет делать декораторы с аргументами без лишних мучений

Создадим декоратор для декораторов

![decorator](yo_dawg.jpg)

In [290]:
def parametrized(dec):
    def layer(*args, **kwargs):
        def repl(f):
            return dec(f, *args, **kwargs)
        return repl
    return layer

Сделаем, к примеру, декоратор, умножающую результат выполнения функции на заданное число

In [292]:
@parametrized
def multiply(f, n):
    def aux(*xs, **kws):
        return n * f(*xs, **kws)
    return aux

@multiply(2)
def function(a):
    return 10 + a

print (function(3))  # Prints 26

@multiply(3)
def function_again(a):
    return 10 + a

print (function(3))          # Keeps printing 26
print (function_again(3))   # Prints 39, namely 3 * (10 + 3)

26
26
39


Ну и интересный пример того, как проверять типы передаваемых функции аргументов из того же ответа. Здесь используются zip и enumerate, которые будут разобраны в следующем занятии.

In [295]:
@parametrized
def types(f, *types):
    def rep(*args):
        for n, (a, t) in enumerate(zip(args, types)):
            if type(a) is not t:
                raise TypeError('Value %d has not type %s. %s instead' %
                    (n, t, type(a))
                )
        return f(*args)
    return rep

@types(str, int)  # arg1 is str, arg2 is int
def string_multiply(text, times):
    return text * times

print(string_multiply('hello', 3))    # prints hellohellohello
print(string_multiply(3, 3))          # Fails miserably with TypeError

hellohellohello


TypeError: Value 0 has not type <class 'str'>. <class 'int'> instead

Для упрощения работы с функциями существуют модуль [functools](https://docs.python.org/3/library/functools.html)

In [306]:
import functools

Например, можно сделать так, чтобы функция хранила результаты последних нескольких своих вызов с помощью **functools.lru_cache**. 
Можем сделать то же, что мы делали подручными средствами. Логично, что будет не хуже, а даже лучше:)


In [334]:
def fibonacci(n):

    f0, f1 = 0, 1
    for i in range(n):
        f0, f1 = f1, f0 + f1
    return f0

def fibonacci_rec_hash(n, calc_memory = {0: 0, 1 : 1}):
    if n < 0:
        return 0
    result = calc_memory.get(n, None) # safe way to get value, 
    # if value doesn't exist, returns default value, here it's None
    if result is not None:
        return result

    result = fibonacci_rec_hash(n - 1) + fibonacci_rec_hash(n - 2)
    calc_memory[n] = result
    return result

@functools.lru_cache(maxsize=1000, typed=False)
def fibonacci_rec(n):
    if n < 0:
        return 0
    
    if n == 0:
        return 0
    if n == 1:
        return 1
    
    return fibonacci_rec(n - 1) + fibonacci_rec(n - 2)

In [331]:
%%timeit
fibonacci_rec_hash(100)

244 ns ± 12.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [333]:
%%timeit
fibonacci_rec(100)

94.6 ns ± 2.6 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [332]:
%%timeit
fibonacci(100)

5.43 µs ± 166 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


В этом модуле есть и несколько других полезных функций, но их мы разберем позже